In [24]:
import pandas as pd
import numpy as np

# Read in the data with the first two lines as a multi-index and the 'Player ID' column as the index
df = pd.read_csv('../../data/interim/nba_draft_data_combined.csv', header=[0,1], index_col=0)

# replace all 0s with NaNs
df = df.replace(0, np.nan)

df.head()

Career    Career                                                           \
Player ID Season Pk   Tm               Player       College   Yrs       G   
947.0       1996  1  PHI        Allen Iverson    Georgetown  14.0   914.0   
948.0       1996  2  TOR         Marcus Camby         UMass  17.0   973.0   
949.0       1996  3  VAN  Shareef Abdur-Rahim    California  12.0   830.0   
950.0       1996  4  MIL      Stephon Marbury  Georgia Tech  13.0   846.0   
951.0       1996  5  MIN            Ray Allen         UConn  18.0  1300.0   

Career                                                                        \
Player ID       MP      PTS     TRB     AST    FG%    3P%    FT%  MP.1 PTS.1   
947.0      37584.0  24368.0  3394.0  5624.0  0.425  0.313  0.780  41.1  26.7   
948.0      28684.0   9262.0  9513.0  1837.0  0.466  0.205  0.670  29.5   9.5   
949.0      28882.0  15028.0  6239.0  2109.0  0.472  0.297  0.810  34.8  18.1   
950.0      31891.0  16297.0  2516.0  6471.0  0.433  0.325  0.784  37.7  19.3   
951.0      46344.0  24505.0  5272.0  4361.0  0.452  0.400  0.894  35.6  18.9   

Career                                         1996-97                         \
Player ID TRB.1 AST.1     WS  WS/48  BPM  VORP     AGE DEF_RATING    GP   MIN   
947.0       3.7   6.2   99.0  0.126  3.2  49.6    22.0      111.4  76.0  40.0   
948.0       9.8   1.9   81.6  0.137  2.3  31.3    23.0      104.3  63.0  30.1   
949.0       7.5   2.5   71.2  0.118  1.0  21.9    20.0      110.4  80.0  35.0   
950.0       3.0   7.6   77.5  0.117  1.8  30.7    20.0      107.6  67.0  34.7   
951.0       4.1   3.4  145.1  0.150  2.9  57.6    21.0      106.4  82.0  31.0   

Career                                                                  \
Player ID NET_RATING OFF_RATING    PIE TEAM_ABBREVIATION       TEAM_ID   
947.0           -7.1      104.3  0.107               PHI  1.610613e+09   
948.0           -0.7      103.6  0.107               TOR  1.610613e+09   
949.0          -11.3       99.1  0.111               VAN  1.610613e+09   
950.0           -3.4      104.2  0.107               MIN  1.610613e+09   
951.0           -1.0      105.4  0.089               MIL  1.610613e+09   

Career    1997-98                                                      \
Player ID     AGE DEF_RATING    GP   MIN NET_RATING OFF_RATING    PIE   
947.0        23.0      104.3  80.0  39.4       -1.9      102.4  0.135   
948.0        24.0      106.6  63.0  31.7       -8.2       98.4  0.084   
949.0        21.0      110.8  82.0  35.9       -7.6      103.2  0.127   
950.0        21.0      105.5  82.0  38.0        0.9      106.4  0.108   
951.0        22.0      103.3  82.0  40.1       -1.1      102.2  0.109   

Career                                    1998-99                         \
Player ID TEAM_ABBREVIATION       TEAM_ID     AGE DEF_RATING    GP   MIN   
947.0                   PHI  1.610613e+09    24.0       94.3  48.0  41.5   
948.0                   TOR  1.610613e+09    25.0       98.3  46.0  20.6   
949.0                   VAN  1.610613e+09    22.0      106.9  50.0  40.4   
950.0                   MIN  1.610613e+09    22.0      100.4  49.0  38.6   
951.0                   MIL  1.610613e+09    23.0       99.9  50.0  34.4   

Career                                                                  \
Player ID NET_RATING OFF_RATING    PIE TEAM_ABBREVIATION       TEAM_ID   
947.0            4.4       98.7  0.154               PHI  1.610613e+09   
948.0           -1.2       97.0  0.111               NYK  1.610613e+09   
949.0           -9.5       97.4  0.138               VAN  1.610613e+09   
950.0            2.7      103.1  0.137               NJN  1.610613e+09   
951.0            3.7      103.6  0.125               MIL  1.610613e+09   

Career    1999-00                                                      \
Player ID     AGE DEF_RATING    GP   MIN NET_RATING OFF_RATING    PIE   
947.0        25.0       99.9  70.0  40.8        2.0      102.0  0.126   
948.0        26.0       97.5 

In [25]:
# Get the position of the last career column
position_to_insert = len(df['Career'].columns)
# Get the list of columns to insert
columns_to_insert = [('Career', 'OFF_RATING'), ('Career', 'DEF_RATING'), ('Career', 'NET_RATING'), ('Career', 'PIE')]

# Insert multiple columns with NaN values
for col in columns_to_insert:
    df.insert(loc=position_to_insert, column=col, value=np.nan)

In [26]:
# create list with all seasons between 1996-97 and 2022-23
seasons = []
for i in range(1996, 2023):
    seasons.append(str(i) + '-' + str(i+1)[2:])

# for each season, insert a new column with the product of 'GP' and 'OFF_RATING'
for season in seasons:
    # find the (season, 'OFF_RATING') column as the position to insert the new column
    position_to_insert = df.columns.get_loc((season, 'OFF_RATING'))
    df.insert(loc=position_to_insert, column=(season, 'OFF_RATING_weighted'), value=np.nan)
    # fill the new column with the product of (season,'GP') and (season,'OFF_RATING')
    df[(season, 'OFF_RATING_weighted')] = df[(season, 'GP')] * df[(season, 'OFF_RATING')]
    
    position_to_insert = df.columns.get_loc((season, 'DEF_RATING'))
    df.insert(loc=position_to_insert, column=(season, 'DEF_RATING_weighted'), value=np.nan)
    df[(season, 'DEF_RATING_weighted')] = df[(season, 'GP')] * df[(season, 'DEF_RATING')]
    
    position_to_insert = df.columns.get_loc((season, 'NET_RATING'))
    df.insert(loc=position_to_insert, column=(season, 'NET_RATING_weighted'), value=np.nan)
    df[(season, 'NET_RATING_weighted')] = df[(season, 'GP')] * df[(season, 'NET_RATING')]
    
    position_to_insert = df.columns.get_loc((season, 'PIE'))
    df.insert(loc=position_to_insert, column=(season, 'PIE_weighted'), value=np.nan)
    df[(season, 'PIE_weighted')] = df[(season, 'GP')] * df[(season, 'PIE')]

/var/folders/zb/fj1ym99j70b38snb8v3_0g580000gn/T/ipykernel_9809/1738369481.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.insert(loc=position_to_insert, column=(season, 'DEF_RATING_weighted'), value=np.nan)
/var/folders/zb/fj1ym99j70b38snb8v3_0g580000gn/T/ipykernel_9809/1738369481.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.insert(loc=position_to_insert, column=(season, 'NET_RATING_weighted'), value=np.nan)
/var/folders/zb/fj1ym99j70b38snb8v3_0g580000gn/T/ipykernel_9809/1738369481.py:23: PerformanceWarning: 

In [27]:
# in the ('Career', rating) column, fill the NaNs with the sum of the weighted columns divided by the ('Career', 'G') column
df[('Career', 'OFF_RATING')] = df.filter(regex='OFF_RATING_weighted').sum(axis=1) / df[('Career', 'G')]
df[('Career', 'DEF_RATING')] = df.filter(regex='DEF_RATING_weighted').sum(axis=1) / df[('Career', 'G')]
df[('Career', 'NET_RATING')] = df.filter(regex='NET_RATING_weighted').sum(axis=1) / df[('Career', 'G')]
df[('Career', 'PIE')] = df.filter(regex='PIE_weighted').sum(axis=1) / df[('Career', 'G')]
# drop the weighted columns
df = df.drop(df.filter(regex='_weighted').columns, axis=1)

In [28]:
# copy the 'Career' columns to a new dataframe
df_career = df['Career'].copy()

# export the dataframe to a csv file
df_career.to_csv('../../data/interim/player_career_avg.csv')